## Circle 13 Chicago Dataset Exploratory Data Analysis
#### This is a circle 13 project for exploratory data analysis for the chicago dataset
#### Members
* Otim William Gerison
* Roddiyyat Nasirudeen Taiwo
* Okafor Osita Brian

### 1. Data Preparation 

In [36]:
#Importing the necessary libraries for EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
file_path = "e:\AltSchool\Circle-13\Chicago Data\crime_data_chicago.csv"

In [38]:
#Reading the csv file into a dataframe
df = pd.read_csv(file_path)

In [39]:
#Previewing the Dataset
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,0,6407111,HP485721,07/26/2008 02:30:00 PM,085XX S MUSKEGON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,...,10.0,46.0,14,1196638.0,1848800.0,2008,02/28/2018 03:56:25 PM,41.739980,-87.555120,"(41.739979622, -87.555120042)"
1,1,11398199,JB372830,07/31/2018 10:57:00 AM,092XX S ELLIS AVE,143C,WEAPONS VIOLATION,UNLAWFUL POSS AMMUNITION,POOL ROOM,True,...,8.0,47.0,15,1184499.0,1843935.0,2018,08/07/2018 04:02:59 PM,41.726922,-87.599747,"(41.726922145, -87.599746995)"
2,2,5488785,HN308568,04/27/2007 10:30:00 AM,062XX N TRIPP AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,True,...,39.0,12.0,05,1146911.0,1941022.0,2007,02/28/2018 03:56:25 PM,41.994138,-87.734959,"(41.994137622, -87.734959049)"
3,3,11389116,JB361368,07/23/2018 08:55:00 AM,0000X N KEELER AVE,0560,ASSAULT,SIMPLE,NURSING HOME/RETIREMENT HOME,False,...,28.0,26.0,08A,1148388.0,1899882.0,2018,07/30/2018 03:52:24 PM,41.881217,-87.730590,"(41.881217483, -87.730589961)"
4,4,12420431,JE297624,07/11/2021 06:40:00 AM,016XX W HARRISON ST,051A,ASSAULT,AGGRAVATED - HANDGUN,PARKING LOT / GARAGE (NON RESIDENTIAL),False,...,27.0,28.0,04A,1165430.0,1897441.0,2021,07/18/2021 04:56:02 PM,41.874174,-87.668082,"(41.874173691, -87.668082118)"


In [40]:
df.shape

(2278726, 23)

In [41]:
#Making a copy of the dataset
df_copy = df.copy()

In [42]:
#Checking the data types of our columns
df_copy.dtypes

Unnamed: 0                int64
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

Converting the Date from object to datetime

In [43]:
df_copy["Date"] = pd.to_datetime(df_copy["Date"])

C:\Users\UNGP\AppData\Local\Temp\ipykernel_3200\1296833322.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy["Date"] = pd.to_datetime(df_copy["Date"])


Checking for missing values

In [44]:
def missing_values(df_copy):
    row, column = df_copy.shape
    return df_copy.isna().sum()

In [45]:
missing_values(df_copy)

Unnamed: 0                   0
ID                           0
Case Number                  1
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      2877
Arrest                       0
Domestic                     0
Beat                         0
District                    12
Ward                    184695
Community Area          184267
FBI Code                     0
X Coordinate             23985
Y Coordinate             23985
Year                         0
Updated On                   0
Latitude                 23985
Longitude                23985
Location                 23985
dtype: int64

Since Ward, Community area, Location, Location Description and District are categorical in nature, we use the mode to fill in the missing values.

In [46]:
#Deriving their modes
ward = df_copy["Ward"].mode()[0]
community_area = df_copy["Community Area"].mode()[0]
location= df_copy["Location"].mode()[0]
loc_description = df_copy["Location Description"].mode()[0]
district = df_copy["District"].mode()[0]

In [47]:
#Filling in the missing values with the mode
df_copy["Ward"] = df_copy["Ward"].fillna(ward)
df_copy["Community Area"] = df_copy["Community Area"].fillna(community_area)
df_copy["Location"] = df_copy["Location"].fillna(location)
df_copy["District"] = df_copy["District"].fillna(district)
df_copy["Location Description"] = df_copy["Location Description"].fillna(loc_description)

There is a missing value in the Case Number column but we can not use median or mode or mean to fill it and since it is only 1, we drop it.

In [48]:
df_copy.dropna(inplace = True)

In [49]:
df_copy.isna().sum()

Unnamed: 0              0
ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
Updated On              0
Latitude                0
Longitude               0
Location                0
dtype: int64

Checking for duplicates

In [50]:
print((df_copy.duplicated()).sum())

0


In [51]:
#Dropping unnecessary columns
def drop_column(df_copy):
    df_copy.drop(columns = ["Unnamed: 0", "ID", "Longitude", "Latitude"], inplace=True)

In [52]:
drop_column(df_copy)

Feature engineering to create new columns for the month, day and season of crime.

In [54]:
def feature_engineering(df_copy):
  #Creating new month and day columns
  df_copy['Month'] = df_copy['Date'].dt.month_name()
  df_copy['Day'] = df_copy['Date'].dt.day_name()

def get_season(month):
  #Mapping month to its corresponding season
  if month in ["December", "January", "February"]:
    return 'Winter'
  elif month in ["March", "April", "May"]:
    return 'Spring'
  elif month in ["June", "July", "August"]:
    return 'Summer'
  else:
    return 'Autumn'

In [55]:
feature_engineering(df_copy)

In [57]:
#Creating a season column
df_copy["Season"] = df_copy["Month"].apply(get_season)

In [58]:
df_copy.head()

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Location,Month,Day,Season
0,HP485721,2008-07-26 14:30:00,085XX S MUSKEGON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,423,...,46.0,14,1196638.0,1848800.0,2008,02/28/2018 03:56:25 PM,"(41.739979622, -87.555120042)",July,Saturday,Summer
1,JB372830,2018-07-31 10:57:00,092XX S ELLIS AVE,143C,WEAPONS VIOLATION,UNLAWFUL POSS AMMUNITION,POOL ROOM,True,False,413,...,47.0,15,1184499.0,1843935.0,2018,08/07/2018 04:02:59 PM,"(41.726922145, -87.599746995)",July,Tuesday,Summer
2,HN308568,2007-04-27 10:30:00,062XX N TRIPP AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,True,False,1711,...,12.0,05,1146911.0,1941022.0,2007,02/28/2018 03:56:25 PM,"(41.994137622, -87.734959049)",April,Friday,Spring
3,JB361368,2018-07-23 08:55:00,0000X N KEELER AVE,0560,ASSAULT,SIMPLE,NURSING HOME/RETIREMENT HOME,False,False,1115,...,26.0,08A,1148388.0,1899882.0,2018,07/30/2018 03:52:24 PM,"(41.881217483, -87.730589961)",July,Monday,Summer
4,JE297624,2021-07-11 06:40:00,016XX W HARRISON ST,051A,ASSAULT,AGGRAVATED - HANDGUN,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1231,...,28.0,04A,1165430.0,1897441.0,2021,07/18/2021 04:56:02 PM,"(41.874173691, -87.668082118)",July,Sunday,Summer


In [ ]:
df_copy.to_csv("dfchicago_clean.csv")

### 2. Statistical Exploration 

In [144]:
df_copy.describe()

,Date,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year
count,2254741,2.254741e+06,2.254741e+06,2.254741e+06,2.254741e+06,2.254741e+06,2.254741e+06,2.254741e+06
mean,2010-02-18 11:09:03.858346752,1.186136e+03,1.128823e+01,2.314287e+01,3.652911e+01,1.164569e+06,1.885747e+06,2.009633e+03
min,2001-01-01 00:00:00,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03
25%,2004-12-17 15:20:00,6.210000e+02,6.000000e+00,1.100000e+01,2.400000e+01,1.152948e+06,1.859053e+06,2.004000e+03
50%,2009-04-24 06:30:00,1.034000e+03,1.000000e+01,2.400000e+01,2.900000e+01,1.166060e+06,1.890673e+06,2.009000e+03
75%,2014-11-17 11:00:00,1.731000e+03,1.700000e+01,3.200000e+01,5.300000e+01,1.176365e+06,1.909219e+06,2.014000e+03
max,2022-07-27 23:35:00,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.022000e+03
std,NaN,7.026159e+02,6.945762e+00,1.333986e+01,2.092605e+01,1.673955e+04,3.209855e+04,6.010493e+00


From the above description, this is an accumulated dataset of 21 years (2001 - 2022).

In [145]:
df_copy['Year'].unique()

array([2008, 2018, 2007, 2021, 2001, 2006, 2014, 2010, 2016, 2011, 2004,
       2012, 2015, 2003, 2002, 2017, 2013, 2020, 2022, 2019, 2005, 2009],
      dtype=int64)

In [146]:
df_copy.loc[:,['Primary Type', 'Location Description', 'Year', 'Season']].head()

,Primary Type,Location Description,Year,Season
0,CRIMINAL DAMAGE,STREET,2008,Summer
1,WEAPONS VIOLATION,POOL ROOM,2018,Summer
2,BURGLARY,RESIDENCE,2007,Spring
3,ASSAULT,NURSING HOME/RETIREMENT HOME,2018,Summer
4,ASSAULT,PARKING LOT / GARAGE (NON RESIDENTIAL),2021,Summer


In [147]:
#mode
df_copy.mode()

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Location,Month,Day,Season
0,HJ104730,2007-01-01 00:01:00,100XX W OHARE ST,0820,THEFT,SIMPLE,STREET,False,False,421.0,...,25.0,06,1100658.0,1934241.0,2001.0,02/10/2018 03:50:01 PM,"(41.976290414, -87.905227221)",July,Friday,Summer
1,HK172551,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#median
df_copy.median(numeric_only=True)

In [ ]:
#mean
df_copy.mean()

In [ ]:
monthly_crime_count = df_copy.groupby('Month')['Primary Type'].value_counts().unstack().fillna(0)

In [ ]:
monthly_crime_count.mean()

From the above analysis, the monthly most crime is THEFT and the least crime is NON-CRIMINAL(SUBJECT SPECIFIED).

In [ ]:
daily_crime_count = df_copy.groupby('Day')['Primary Type'].value_counts

In [ ]:
daily_crime_count().head()

From the above analysis, The crime with the Highest daily occurrence is THEFT and the day of the week with the highest number of crimes is Friday.

In [73]:
daily_crime_count().tail(1)

Day        Primary Type  
Wednesday  NON - CRIMINAL    1
Name: count, dtype: int64

From the above analysis, The crime with the least daily occurrence is NON-CRIMINAL and the day of the week with the least number of crimes is Wednesday.

In [66]:
seasonal_crime_count = df_copy.groupby('Season')['Primary Type'].value_counts

In [69]:
seasonal_crime_count().head()

Season  Primary Type   
Autumn  THEFT              120963
        BATTERY             99927
        CRIMINAL DAMAGE     65390
        NARCOTICS           53364
        ASSAULT             35703
Name: count, dtype: int64

From the above analysis, The crime with the Highest seasonal occurrence is THEFT and the season with the highest number of crimes is Autumn.

In [74]:
seasonal_crime_count().tail()

Season  Primary Type                    
Winter  NON-CRIMINAL                        6
        OTHER NARCOTIC VIOLATION            4
        NON - CRIMINAL                      3
        RITUALISM                           3
        NON-CRIMINAL (SUBJECT SPECIFIED)    2
Name: count, dtype: int64

From the above analysis, The crime with the least saesonal occurrence is NON-CRIMINAL and the season with the least number of crimes is Winter.

In [75]:
highest_crime_type = df_copy['Primary Type'].value_counts().head(1)

In [78]:
highest_crime_type

Primary Type
THEFT    473443
Name: count, dtype: int64

THEFT is the crime type with highest occurrence.

In [80]:
lowest_crime_type = df_copy['Primary Type'].value_counts().tail(1)

In [81]:
lowest_crime_type

Primary Type
NON-CRIMINAL (SUBJECT SPECIFIED)    2
Name: count, dtype: int64

NON-CRIMINAL the crime type with lowest occurrence

### 3. Visual Exploration 